In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
import torch
from typing import Literal, TypedDict

In [2]:
sys.path.append("..")
os.environ["USER_PATH"] = "../userdata/"

In [3]:
from system.namespace.store import get_namespace

In [4]:
namespace = get_namespace("train_db")

In [5]:
from system.embedding.store import get_embed_store
from misc.lru import LRU

In [6]:
embeds = get_embed_store(namespace)

In [7]:
role_c = "child"
role_p = "parent"

In [8]:
total_c = embeds.get_embedding_count(role_c)
total_c

689414

In [9]:
total_p = embeds.get_embedding_count(role_p)
total_p

689414

In [10]:
from sklearn.manifold import TSNE

In [11]:
%%time

x_0 = np.vstack([
    embed[1].ravel().detach().numpy()
    for embed in embeds.get_all_embeddings(role_c, progress_bar=True)
])
y_0 = x_0.shape[0]
x_1 = np.vstack([
    embed[1].ravel().detach().numpy()
    for embed in embeds.get_all_embeddings(role_p, progress_bar=True)
])
y_1 = x_1.shape[0]
y = [
    "tab:blue" for _ in range(y_0)
] + [
    "tab:orange" for _ in range(y_1)
]
x = np.vstack([x_0, x_1])
x.shape

  0%|          | 0/689414 [00:00<?, ?it/s]

  0%|          | 0/689414 [00:00<?, ?it/s]

CPU times: user 3min 35s, sys: 44.3 s, total: 4min 19s
Wall time: 8min 38s


(1378828, 768)

In [12]:
x[:2, :]

array([[-0.06251884,  0.06813351,  0.25403288, ...,  0.13894862,
        -0.40365079, -0.04747996],
       [-0.0618586 ,  0.06744971,  0.23952982, ...,  0.14076276,
        -0.40367034, -0.04754893]])

In [ ]:
%%time

x_embedded = TSNE(
    n_components=2,
    learning_rate="auto",
    init="random",
    # perplexity=10,
    method="barnes_hut",
    random_state=42,
    n_jobs=-1).fit_transform(x)

In [ ]:
pd.DataFrame(x_embedded).plot.scatter(x=0, y=1, s=0.5, c=y, xlabel="", ylabel="")